### MNIST Example with PyTorch

Let's be nice and train a neural network on MNIST .

We will train our model using 2 different approach to show case each of them .
First a Logistic Regression approach and then use a Neural Network

In [16]:

import numpy as np

import torch
from torch.autograd import Variable
from torch import optim


import gzip
import os
from os import path
import numpy as np

import sys
if sys.version_info.major < 3:
    import urllib
else:
    import urllib.request as request


DATASET_DIR = '../data'

MNIST_FILES = ["train-images-idx3-ubyte.gz", "train-labels-idx1-ubyte.gz",
               "t10k-images-idx3-ubyte.gz", "t10k-labels-idx1-ubyte.gz"]




def one_hot(x, n):
    if type(x) == list:
        x = np.array(x)
    x = x.flatten()
    o_h = np.zeros((len(x), n))
    o_h[np.arange(len(x)), x] = 1
    return o_h


def load_mnist(ntrain=60000, ntest=10000, onehot=True):
    data_dir = os.path.join(DATASET_DIR)
    
    with gzip.open(os.path.join(data_dir, 'train-images-idx3-ubyte.gz')) as fd:
        buf = fd.read()
        loaded = np.frombuffer(buf, dtype=np.uint8)
        trX = loaded[16:].reshape((60000, 28 * 28)).astype(float)

    with gzip.open(os.path.join(data_dir, 'train-labels-idx1-ubyte.gz')) as fd:
        buf = fd.read()
        loaded = np.frombuffer(buf, dtype=np.uint8)
        trY = loaded[8:].reshape((60000))

    with gzip.open(os.path.join(data_dir, 't10k-images-idx3-ubyte.gz')) as fd:
        buf = fd.read()
        loaded = np.frombuffer(buf, dtype=np.uint8)
        teX = loaded[16:].reshape((10000, 28 * 28)).astype(float)

    with gzip.open(os.path.join(data_dir, 't10k-labels-idx1-ubyte.gz')) as fd:
        buf = fd.read()
        loaded = np.frombuffer(buf, dtype=np.uint8)
        teY = loaded[8:].reshape((10000))

    trX /= 255.
    teX /= 255.

    trX = trX[:ntrain]
    trY = trY[:ntrain]

    teX = teX[:ntest]
    teY = teY[:ntest]

    if onehot:
        trY = one_hot(trY, 10)
        teY = one_hot(teY, 10)
    else:
        trY = np.asarray(trY)
        teY = np.asarray(teY)

    return trX, teX, trY, teY

def build_model(input_dim, output_dim):
    model = torch.nn.Sequential()
    model.add_module("linear",
                     torch.nn.Linear(input_dim, output_dim, bias=False))
    return model


def train(model, loss, optimizer, x_val, y_val):
    x = Variable(x_val, requires_grad=False)
    y = Variable(y_val, requires_grad=False)

    # Reset gradient
    optimizer.zero_grad()

    # Forward
    fx = model.forward(x)
    output = loss.forward(fx, y)

    # Backward
    output.backward()

    # Update parameters
    optimizer.step()

    return output.data[0]


def predict(model, x_val):
    x = Variable(x_val, requires_grad=False)
    output = model.forward(x)
    return output.data.numpy().argmax(axis=1)


def main():
    torch.manual_seed(42)
    trX, teX, trY, teY = load_mnist(onehot=False)
    trX = torch.from_numpy(trX).float()
    teX = torch.from_numpy(teX).float()
    trY = torch.from_numpy(trY).long()

    n_examples, n_features = trX.size()
    n_classes = 10
    model = build_model(n_features, n_classes)
    loss = torch.nn.CrossEntropyLoss(size_average=True)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    batch_size = 100

    for i in range(100):
        cost = 0.
        num_batches = n_examples // batch_size
        for k in range(num_batches):
            start, end = k * batch_size, (k + 1) * batch_size
            cost += train(model, loss, optimizer,
                          trX[start:end], trY[start:end])
        predY = predict(model, teX)
        print("Epoch %d, cost = %f, acc = %.2f%%"
              % (i + 1, cost / num_batches, 100. * np.mean(predY == teY)))


if __name__ == "__main__":
    main()

Epoch 1, cost = 0.548300, acc = 90.01%
Epoch 2, cost = 0.364975, acc = 90.91%
Epoch 3, cost = 0.338107, acc = 91.34%
Epoch 4, cost = 0.324099, acc = 91.48%
Epoch 5, cost = 0.315061, acc = 91.71%
Epoch 6, cost = 0.308569, acc = 91.80%
Epoch 7, cost = 0.303588, acc = 91.86%
Epoch 8, cost = 0.299590, acc = 91.92%
Epoch 9, cost = 0.296276, acc = 91.95%
Epoch 10, cost = 0.293460, acc = 92.01%
Epoch 11, cost = 0.291021, acc = 92.03%
Epoch 12, cost = 0.288878, acc = 92.01%
Epoch 13, cost = 0.286971, acc = 92.05%
Epoch 14, cost = 0.285256, acc = 92.09%
Epoch 15, cost = 0.283702, acc = 92.14%
Epoch 16, cost = 0.282282, acc = 92.18%
Epoch 17, cost = 0.280978, acc = 92.19%
Epoch 18, cost = 0.279773, acc = 92.20%
Epoch 19, cost = 0.278654, acc = 92.22%
Epoch 20, cost = 0.277611, acc = 92.23%
Epoch 21, cost = 0.276635, acc = 92.24%
Epoch 22, cost = 0.275718, acc = 92.26%
Epoch 23, cost = 0.274855, acc = 92.25%
Epoch 24, cost = 0.274040, acc = 92.29%
Epoch 25, cost = 0.273268, acc = 92.30%
Epoch 26,

Let's see if we can imporve accuracy using a feed-forward network 

In [18]:
#all we have to do is modify the build model function

def build_model(input_dim, output_dim):
    model = torch.nn.Sequential()
    model.add_module("linear_1", torch.nn.Linear(input_dim, 512, bias=False))
    model.add_module("sigmoid_1", torch.nn.Sigmoid())
    model.add_module("linear_2", torch.nn.Linear(512, output_dim, bias=False))
    return model

main()

Epoch 1, cost = 1.076818, acc = 87.04%
Epoch 2, cost = 0.429945, acc = 89.41%
Epoch 3, cost = 0.362955, acc = 90.13%
Epoch 4, cost = 0.335733, acc = 90.76%
Epoch 5, cost = 0.319703, acc = 91.24%
Epoch 6, cost = 0.308471, acc = 91.51%
Epoch 7, cost = 0.299678, acc = 91.67%
Epoch 8, cost = 0.292222, acc = 91.84%
Epoch 9, cost = 0.285509, acc = 92.02%
Epoch 10, cost = 0.279178, acc = 92.26%
Epoch 11, cost = 0.273001, acc = 92.42%
Epoch 12, cost = 0.266836, acc = 92.60%
Epoch 13, cost = 0.260599, acc = 92.70%
Epoch 14, cost = 0.254260, acc = 92.91%
Epoch 15, cost = 0.247827, acc = 93.04%
Epoch 16, cost = 0.241340, acc = 93.24%
Epoch 17, cost = 0.234855, acc = 93.46%
Epoch 18, cost = 0.228430, acc = 93.57%
Epoch 19, cost = 0.222120, acc = 93.75%
Epoch 20, cost = 0.215968, acc = 93.89%
Epoch 21, cost = 0.210001, acc = 94.03%


KeyboardInterrupt: 

By Epoch 21 we reached a 94% accuracy improving upon logistic regression by 2%